In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df_train = pd.read_csv('E:\\PycharmProjects\\Kaggle\\Titanic\\df_train.csv')
df_test = pd.read_csv('E:\\PycharmProjects\\Kaggle\\Titanic\\df_test.csv')

## 2. Building predicion model

### 2.1. Logistic regression

Since our goal is to predict 1 of 2 output results logistic regression model is my first choice.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
drop_columns=['PassengerId', 'Survived']
X = df_train.drop(columns=drop_columns)
y=df_train['Survived']

In [ ]:
X_train = StandardScaler().fit_transform(X)
y_train = df_train['Survived'].values
X_test = StandardScaler().fit_transform(df_test.drop(columns=drop_columns))

print('X_train shape: {}'.format(X_train.shape))
print('Y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(X_test.shape))

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

y_pred = logmodel.predict(X_test)

acc_log = round(logmodel.score(X_train, y_train) * 100, 2)
acc_log

In [ ]:
prediction = logmodel.predict(X_test)

In [ ]:
# print(classification_report(y_test, prediction))

In [ ]:
# print(confusion_matrix(y_test, prediction))

##### Let's apply our trained model on Kaggle's test set

In [ ]:
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": y_pred
    })

submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()
